In [13]:
import mdtraj as md
import itertools, math
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

names = ['QPM1_ACN_K','QPM1_ACN_Na','QPM1_H2O_K','QPM1_H20_Na','QPM3_ACN_K','QPM3_ACN_Na','QPM3_H2O_K','QPM3_H20_Na']
kB = 1.38064852*10**-23
T = 300
ion_names = ['K','NA']
name_count = 0
for macrocycle in ['0','1']:
    for solvent in ['a','w']:
        for x,ion in enumerate(['k','n']):
            prefix = f'{macrocycle}_{solvent}_{ion}'
            traj = md.load(f'{prefix}.trr',top=f'{prefix}.gro')
            carbonyl_o_indices = [a.index for a in traj.topology.atoms if a.name == 'O'][:6]
            ion_indices = [a.index for a in traj.topology.atoms if a.name == ion_names[x]]
            pairs = itertools.product(carbonyl_o_indices, ion_indices)
            ion_O_distances = np.concatenate(md.compute_distances(traj, pairs))
            counts, distances = np.histogram(ion_O_distances, 1000)
            counts = [x+1 for x in counts]
            pmf = [-kB*T*math.log(counts[i]) + kB*T*2*distances[i] for i in range(len(counts))]
            ndx = min(range(len(distances)), key=lambda i: abs(distances[i]-2.0))
            print(ndx)
            pmf = [x - pmf[ndx] for x in pmf]
            plt.figure(2, figsize=(10,5))
            if solvent == 'w':
                plt.plot(distances[1:],pmf,label=names[name_count], linestyle='dotted')
            else:
                plt.plot(distances[1:],pmf,label=names[name_count])
            name_count += 1

plt.xlim(0.15,2.0)
plt.ylabel('PMF (kT)')
plt.xlabel('Distance of Ion to Macrocycle Center (nm)')
plt.title('PMFs Comparing Solvent, Cation, and Macrocycle Substituents')
plt.legend()
plt.tight_layout()
plt.savefig('pmf.pdf')
plt.close()

637
694
509
515
628
637
541
546


In [1]:
a = [0,1,2,3,4]
a.index(2)

2